# Modelling the Solar System

In [1]:
from vpython import *
import numpy as np
import matplotlib. pyplot as plt

<IPython.core.display.Javascript object>

### Scene set-up for modelling

In [2]:
scene = canvas()

<IPython.core.display.Javascript object>

### Defining constants and calculating initial conditions

In [3]:
G = 6.6704e-11

#Defining the acceleration on one body caused by one other body
def accel(a,b):
    rel_pos = b.pos - a.pos
    return G*b.mass*norm(rel_pos)/rel_pos.mag2 

#Summing the accelerations caused by other bodies on any one body
def totalaccel(a, planets):
    sum_accel = vec(0,0,0)
    for b in planets:
        if (a!=b):
            sum_accel = sum_accel + accel(a,b)
    return sum_accel

#Define constants, position, velocity of each planet and sun 
AU = 149.6e9       
G=6.673e-11

#Mass
sun_mass = 2e30
earth_mass = 6e24
moon_mass = 7.35e22
mercury_mass = 3.28e23
venus_mass = 4.87e24
mars_mass = 6.39e23
saturn_mass = 5.7e26
uranus_mass = 8.7e25
neptune_mass = 1e26
pluto_mass = 1.3e22

#Raduis of orbit
mercury_dist = 0.397*AU
venus_dist = 0.7*AU
mars_dist = 1.5*AU
saturn_dist = 9.5*AU
jupiter_dist = 5.2*AU
uranus_dist = 19.8*AU
neptune_dist = 30*AU
pluto_dist = 39.5*AU
jupiter_mass=1.9e27


#Velocity of planets
earth_vel = 2* np.pi * AU/(365.25 *24. *60.*60.) 
jupiter_vel=2* np.pi *AU*5.2/(11.86*365.25*24.*60.*60)
moon_velx = (2* np.pi *3.84e8/(29.5*24.*60.*60)) + (2* np.pi * AU/(365.25 *24. *60.*60.))
mercury_vel = 2* np.pi * 0.397*AU/(88 *24. *60.*60.) 
venus_vel = 2* np.pi * 0.7*AU/(225 *24. *60.*60.) 
mars_vel = 2* np.pi * 1.5*AU/(687 *24. *60.*60.) 
saturn_vel = 2* np.pi * 9.5*AU/(29*365.25 *24. *60.*60.) 
uranus_vel = 2* np.pi * 19.8*AU/(84*365.25 *24. *60.*60.) 
neptune_vel = 2* np.pi * 30*AU/(165*365.25 *24. *60.*60.) 
pluto_vel = 2* np.pi * 39.5*AU/(248*365.25 *24. *60.*60.) 


In [4]:
scene.background = color.black
scene.autoscale = 0
scene.range = 10*AU

In [5]:
#Define the planets
sun = sphere(pos= vector(0,0,0), velocity = vector(0,0,0),
             mass=sun_mass, radius = 0.05*AU, color =color.yellow)
earth = sphere(pos= vector(AU, 0, 0), velocity = vector(0,earth_vel,0),
               mass=earth_mass, radius=0.05*AU, color =color.cyan)
jupiter = sphere(pos=vector(5.2*AU,0,0),velocity=vector(0,jupiter_vel,0),
                 mass=jupiter_mass, radius=0.05*AU, color=color.red)
mercury = sphere(pos=vector(0.397*AU,0,0),velocity=vector(0,mercury_vel,0),
                 mass=mercury_mass, radius=0.05*AU, color=color.red)
venus = sphere(pos=vector(0.7*AU,0,0),velocity=vector(0,venus_vel,0),
                 mass=venus_mass, radius=0.05*AU, color=color.yellow)
mars = sphere(pos=vector(1.5*AU,0,0),velocity=vector(0,mars_vel,0),
                 mass=mars_mass, radius=0.05*AU, color=color.red)
saturn = sphere(pos=vector(9.5*AU,0,0),velocity=vector(0,saturn_vel,0),
                 mass=saturn_mass, radius=0.05*AU, color=color.orange)
uranus = sphere(pos=vector(19.8*AU,0,0),velocity=vector(0,uranus_vel,0),
                 mass=uranus_mass, radius=0.05*AU, color=color.magenta)
neptune = sphere(pos=vector(30*AU,0,0),velocity=vector(0,neptune_vel,0),
                 mass=neptune_mass, radius=0.05*AU, color=color.blue)
pluto = sphere(pos=vector(39.5*AU,0,0),velocity=vector(0,pluto_vel,0),
                 mass=pluto_mass, radius=0.05*AU, color=color.cyan)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
#Put planets in the planet list
planets = [sun, earth, jupiter, mercury, venus, mars, saturn, uranus, neptune, pluto]

for b in planets:
    b.accel = vector(0,0,0)
    b.track=curve (color = b.color)

In [7]:
#Set momentum in the centre of mass to zero, by summing the momenta of all other bodies and calculating velocity of 
#the sun accordingly.

sum=vector(0,0,0)
for b in planets:
    if (b!=sun):
        sum=sum+b.mass*b.velocity

sun.velocity=-sum/sun.mass

#Define the time step dt
dt=60.*60.*20

In [8]:
#First we calculate the total initial energy in the system by adding kinetic and potential energy
KE_i = 0
PE_i = 0
E_i = 0
for i in planets:
    KE_i = KE_i + (0.5*i.mass*i.velocity.mag2)      #Adding the kinetic energy of each body
    for j in planets:
        if (j != i):
            distance = j.pos - i.pos
            PE_i = PE_i + (6.67e-11*i.mass*j.mass)/(distance).mag  #Adding the gravitational potential energy of each
    E_i = E_i + KE_i + PE_i       #Adding kinetic and potential to get the total initial energy
    
print("Initial Kinetic energy=", KE_i)
print("Initial Potential energy=", PE_i)
print("Total initial energy=", E_i)

Initial Kinetic energy= 1.990481818287155e+35
Initial Potential energy= 7.980179102667262e+35
Total initial energy= 8.381986451520447e+36


### Leapfrog Calculations

In [9]:
#Initializing Leapfrog
for b in planets:
    b.velocity = b.velocity + totalaccel(b, planets)*dt/2.0

counter = 0

#start leap-frog
while True:
    rate(50)  
    for b in planets:
        b.pos = b.pos + b.velocity*dt
        b.track.append(pos=b.pos)

        b.velocity = b.velocity + totalaccel(b, planets)*dt
    
    counter = counter + 1

    scene.center = vector(0,0,0) #view centered on the origin 


KeyboardInterrupt: 

### Checking conservation of energy to estimate error

In [ ]:
#Calculate the total final energy in the system
KE_f = 0
PE_f= 0
E_f = KE_f + PE_f
for i in planets:
    KE_f = KE_f + (0.5*i.mass*i.velocity.mag2)      #Adding the kinetic energy of each
    for j in planets:
        if (j != i):
            distance = j.pos - i.pos
            PE_f = PE_f + (6.67e-11*i.mass*j.mass)/(distance).mag #Adding gravitational potential energy of each body
    E_f = E_f + KE_f + PE_f           #Calculating total final energy as the sum of final kinetic and potential energy
    
print("Final Kinetic energy=", KE_f)
print("Final Potential energy=", PE_f)
print("Total Final energy=", E_f)
print("Difference in total energy is ", ((E_f - E_i)*100)/E_i, "%")